<a href="https://colab.research.google.com/github/libra3910/diveintocode-ml/blob/master/RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### IMDBの取り込み

In [ ]:
# IMDBをカレントフォルダにダウンロード
#!wget http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
# 解凍
#!tar zxf aclImdb_v1.tar.gz
# aclImdb/train/unsupはラベル無しのため削除
#!rm -rf aclImdb/train/unsup
# IMDBデータセットの説明を表示
#!cat aclImdb/README

In [ ]:
from sklearn.datasets import load_files
train_review = load_files('./aclImdb_v1/aclImdb/train/', encoding='utf-8')
x_train, y_train = train_review.data, train_review.target
test_review = load_files('./aclImdb_v1/aclImdb/test/', encoding='utf-8')
x_test, y_test = test_review.data, test_review.target
# ラベルの0,1と意味の対応の表示
print(train_review.target_names)

['neg', 'pos', 'unsup']


### 【問題1】BoWのスクラッチ実装

In [ ]:
import pandas as pd
import numpy as np
# フレーズの代入
phrase = []
phrase.append('This movie is SOOOO funny!!!')
phrase.append('What a movie! I never')
phrase.append('best movie ever!!!!! this movie')

# BOW:1-gram
bow1 =[]
all_bows = []
for i in range(len(phrase)):
    bow1.append(phrase[i].split(' '))
all_bows = np.array(bow1).reshape(-1)
all_bows = np.unique(all_bows)

all_bows_count = []
for j in range(all_bows.shape[0]):
    all_bows_count.append((all_bows[j], bow1[0].count(all_bows[j]), bow1[1].count(all_bows[j]), bow1[2].count(all_bows[j])))

print("BOW:1-gram")
print("１．元のフレーズを表示：")
print(phrase)
print("２．フレーズを分割：")
print(bow1)
print("３．単語の列を作成：")
print(all_bows)
print("４．フレーズごとの単語の出現回数：")
print(pd.DataFrame(np.array(all_bows_count).reshape(-1,4).T))

# BOW:2-gram
bow2 = []
for j in range(len(bow1)):
    tmp = []
    for k in range(len(bow1[j])-1):
        tmp.append(bow1[j][k] + ' ' + bow1[j][k+1])
    bow2.append(tmp)
all_bows2 = np.array(bow2).reshape(-1)
all_bows2 = np.unique(all_bows2)

all_bows_count2 = []
for j in range(all_bows2.shape[0]):
    all_bows_count2.append((all_bows2[j], bow2[0].count(all_bows2[j]), bow2[1].count(all_bows2[j]), bow2[2].count(all_bows2[j])))

print()
print("BOW:2-gram")
print("１．元のフレーズを表示：")
print(phrase)
print("２．フレーズを分割：")
print(bow2)
print("３．単語の列を作成：")
print(all_bows2)
print("４．フレーズごとの単語の出現回数：")
print(pd.DataFrame(np.array(all_bows_count2).reshape(-1,4).T))

BOW:1-gram
１．元のフレーズを表示：
['This movie is SOOOO funny!!!', 'What a movie! I never', 'best movie ever!!!!! this movie']
２．フレーズを分割：
[['This', 'movie', 'is', 'SOOOO', 'funny!!!'], ['What', 'a', 'movie!', 'I', 'never'], ['best', 'movie', 'ever!!!!!', 'this', 'movie']]
３．単語の列を作成：
['I' 'SOOOO' 'This' 'What' 'a' 'best' 'ever!!!!!' 'funny!!!' 'is' 'movie'
 'movie!' 'never' 'this']
４．フレーズごとの単語の出現回数：
  0      1     2     3  4     5          6         7   8      9       10  \
0  I  SOOOO  This  What  a  best  ever!!!!!  funny!!!  is  movie  movie!   
1  0      1     1     0  0     0          0         1   1      1       0   
2  1      0     0     1  1     0          0         0   0      0       1   
3  0      0     0     0  0     1          1         0   0      2       0   

      11    12  
0  never  this  
1      0     0  
2      1     0  
3      0     1  

BOW:2-gram
１．元のフレーズを表示：
['This movie is SOOOO funny!!!', 'What a movie! I never', 'best movie ever!!!!! this movie']
２．フレーズを分割：
[['This movie

### 【問題2】TF-IDFの計算

In [ ]:
x_train[0]

'Full of (then) unknown actors TSF is a great big cuddly romp of a film.<br /><br />The idea of a bunch of bored teenagers ripping off the local sink factory is odd enough, but add in the black humour that Forsyth & Co are so good at and your in for a real treat.<br /><br />The comatose van driver by itself worth seeing, and the canal side chase is just too real to be anything but funny.<br /><br />And for anyone who lived in Glasgow it\'s a great "Oh I know where that is" film.'

In [ ]:
# 後で、この出力結果を問題４にも使用する
from sklearn.feature_extraction.text import TfidfVectorizer
corpus = []
s_cnt = 0
i = 0
while s_cnt < 5000:
    corpus.append(x_train[i])
    vectorizer = TfidfVectorizer()
    X = vectorizer.fit_transform(corpus) # １.corpusが元のフレーズ
    s_cnt += len(vectorizer.get_feature_names()) # ２.フレーズを分割、３.単語の列を作成
    i = i + 1
print(s_cnt)
print(i)

5252
10


In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

# ３．単語の列を作成

sample = []
# ベクトル化する文字列
for i in range(9):
    sample.append(x_train[i])
sample = np.array(sample)

# TfidfVectorizer
vec_tfidf = TfidfVectorizer()

# ベクトル化
X = vec_tfidf.fit_transform(sample)

print('Vocabulary size: {}'.format(len(vec_tfidf.vocabulary_)))
print('Vocabulary content: {}'.format(vec_tfidf.vocabulary_))

Vocabulary size: 859
Vocabulary content: {'full': 288, 'of': 514, 'then': 732, 'unknown': 787, 'actors': 15, 'tsf': 774, 'is': 386, 'great': 312, 'big': 80, 'cuddly': 169, 'romp': 613, 'film': 270, 'br': 96, 'the': 729, 'idea': 359, 'bunch': 104, 'bored': 89, 'teenagers': 721, 'ripping': 602, 'off': 515, 'local': 442, 'sink': 666, 'factory': 250, 'odd': 513, 'enough': 234, 'but': 106, 'add': 19, 'in': 368, 'black': 84, 'humour': 356, 'that': 727, 'forsyth': 283, 'co': 133, 'are': 47, 'so': 670, 'good': 307, 'at': 51, 'and': 35, 'your': 858, 'for': 277, 'real': 579, 'treat': 768, 'comatose': 137, 'van': 798, 'driver': 213, 'by': 107, 'itself': 396, 'worth': 847, 'seeing': 637, 'canal': 112, 'side': 658, 'chase': 126, 'just': 405, 'too': 760, 'to': 754, 'be': 61, 'anything': 43, 'funny': 289, 'anyone': 42, 'who': 831, 'lived': 440, 'glasgow': 302, 'it': 392, 'oh': 516, 'know': 417, 'where': 827, 'amount': 32, 'disappointment': 201, 'am': 27, 'getting': 297, 'these': 734, 'days': 174, 'mo

In [ ]:
# ４．フレーズごとの単語の出現回数　&　５.ベクトル化
pd.DataFrame(X.toarray(), columns=vec_tfidf.get_feature_names())

,10,1930s,1935,1949,1968,1999,2050s,30,45,70,...,would,write,written,wrong,wrote,yellow,york,you,young,your
0,0.00000,0.00000,0.00000,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000,0.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,0.077097
1,0.00000,0.00000,0.00000,0.000000,0.00000,0.00000,0.000000,0.065524,0.065524,0.00000,...,0.042516,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.151750,0.00000,0.000000
2,0.00000,0.00000,0.00000,0.000000,0.00000,0.00000,0.045329,0.000000,0.000000,0.00000,...,0.000000,0.000000,0.038285,0.000000,0.045329,0.000000,0.00000,0.000000,0.00000,0.000000
3,0.00000,0.00000,0.00000,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000,0.00000,...,0.000000,0.073275,0.000000,0.000000,0.000000,0.000000,0.00000,0.200920,0.00000,0.000000
4,0.00000,0.00000,0.00000,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000,0.00000,...,0.000000,0.000000,0.068313,0.000000,0.000000,0.000000,0.00000,0.187316,0.00000,0.052480
5,0.00000,0.00000,0.00000,0.168698,0.00000,0.00000,0.000000,0.000000,0.000000,0.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000
6,0.03251,0.03251,0.03251,0.000000,0.03251,0.03251,0.000000,0.000000,0.000000,0.03251,...,0.042188,0.027458,0.000000,0.000000,0.000000,0.000000,0.03251,0.000000,0.03251,0.000000
7,0.00000,0.00000,0.00000,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000,0.00000,...,0.174033,0.000000,0.000000,0.053643,0.000000,0.053643,0.00000,0.155292,0.00000,0.034807
8,0.00000,0.00000,0.00000,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000,0.00000,...,0.047808,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.085320,0.00000,0.047808


### 【問題3】TF-IDFを用いた学習

In [ ]:
X = pd.DataFrame(X.toarray(), columns=vec_tfidf.get_feature_names())

In [ ]:
X.shape

(9, 859)

In [ ]:
y = []
for i in range(len(X)):
    y.append(y_train[i])

In [ ]:
y = pd.DataFrame(y)

In [ ]:
from sklearn.model_selection import train_test_split
X_train_train, X_train_val, y_train_train, y_train_val = train_test_split(X, y, test_size = 0.3, random_state = 43)

In [ ]:
# ランダムフォレストによる分類器で学習
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier() # ランダムフォレストモデルのインスタンスを作成
rf.fit(X_train_train, y_train_train) # ランダムフォレストモデルの重みを学習

<ipython-input-13-8f6fd8adce3c>:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf.fit(X_train_train, y_train_train) # ランダムフォレストモデルの重みを学習


RandomForestClassifier()

In [ ]:
y_pred = rf.predict(X_train_val)

In [ ]:
print("予測")
print(y_pred.reshape(-1,1))
print("実測")
print(y_train_val)

予測
[[2]
 [2]
 [2]]
実測
   0
3  1
8  2
6  2


### 【問題4】TF-IDFのスクラッチ実装

In [ ]:
# 後で、この出力結果を問題４にも使用する
from sklearn.feature_extraction.text import TfidfVectorizer
corpus = []
s_cnt = 0
i = 0
while s_cnt < 5000:
    corpus.append(x_train[i])
    vectorizer = TfidfVectorizer()
    X = vectorizer.fit_transform(corpus) # １.corpusが元のフレーズ
    s_cnt += len(vectorizer.get_feature_names()) # ２.フレーズを分割、３.単語の列を作成
    i = i + 1
print(s_cnt)

5252


In [ ]:
import re
bow3 = ''
all_bows3 = []
for i in range(10):
    x_train2 = x_train[i].lower() #大文字を小文字に変換する。【問題5】コーパスの前処理をしておく
    x_train2 = re.sub(r'\W',' ', x_train2) # アルファベット、アンダーバー、数字以外の文字を空白に置き換える。
    x_train2 = re.sub(r'\s+',' ',x_train2) # 垂直タブ以外のすべての空白文字が1回以上繰り返す場合、1個の空白に置き換える。
    bow3 += x_train2
bow3 = bow3.split(' ')
all_bows3 = np.array(bow3).reshape(-1)
all_bows3 = np.unique(all_bows3)

In [ ]:
all_bows3.shape

(894,)

In [ ]:
# ４.単語ごとの出現回数
all_bows_count3 = []
for j in range(len(all_bows3)):
    all_bows_count3.append((all_bows3[j], x_train[0].count(all_bows3[j]), x_train[1].count(all_bows3[j]), x_train[2].count(all_bows3[j]), 
                            x_train[3].count(all_bows3[j]), x_train[4].count(all_bows3[j]), x_train[5].count(all_bows3[j]), 
                            x_train[6].count(all_bows3[j]), x_train[7].count(all_bows3[j]), x_train[8].count(all_bows3[j]), 
                            x_train[9].count(all_bows3[j])))


In [ ]:
pd.DataFrame(all_bows_count3)

,0,1,2,3,4,5,6,7,8,9,10
0,,482,1193,1907,808,941,837,2985,1589,1013,331
1,10this,0,0,0,0,0,0,0,0,0,0
2,1930s,0,0,0,0,0,0,1,0,0,0
3,1935,0,0,0,0,0,0,1,0,0,0
4,1949,0,0,0,0,0,2,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
889,yellow,0,0,0,0,0,0,0,1,0,0
890,york,0,0,0,0,0,0,0,0,0,0
891,you,1,4,0,3,4,0,1,6,3,0
892,young,0,0,0,0,0,0,1,0,0,0


In [ ]:
# TF-IDF(索引語頻度逆文書頻度)を求める : TF * IDF
# TF:指定単語の出現頻度 = 文書内の指定単語の出現回数 / 文書内の全単語の出現回数
# IDF:指定単語のレア度 = log(総文書数 / 指定単語を含む文書数)

In [ ]:
all_bows_count3 = np.array(all_bows_count3)

In [ ]:
all_bows_count3.shape

(894, 11)

In [ ]:
# TF, IDFの初期設定
TF = np.zeros((all_bows_count3.shape[0], all_bows_count3.shape[1]))
IDF = np.zeros((all_bows_count3.shape[0], all_bows_count3.shape[1]))

In [ ]:
# 各フレーズのTFを計算
for i in range(1, all_bows_count3.shape[1]):
    for j in range(all_bows_count3.shape[0]):
        TF[j][i]  = all_bows_count3[j][i].astype(int) / np.sum(all_bows_count3[:,i].astype(int))

In [ ]:
# 指定単語ごとのIDFを計算
for i in range(all_bows_count3.shape[0]):
    w_count = 0
    for j in range(1, all_bows_count3.shape[1]):
        if all_bows_count3[i][j].astype(int) != 0:
            w_count += 1
    if w_count == 0:
        w_count +=0.001 # なぜか指定単語を含む文書数=0の文字列があるため、ゼロdivide防止のため、0.001を挿入
    IDF[i] = np.log((all_bows_count3.shape[1] - 1) / w_count)

In [ ]:
# TFとIDFを掛け合わせる。
TF_IDF = TF * IDF
TF_IDF

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.00048199, 0.0007605 , ..., 0.00085024, 0.00065807,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.00123823, 0.        , ..., 0.00036404, 0.00056352,
        0.        ]])

### 【問題5】コーパスの前処理

In [ ]:
import re
x_train2 = x_train[i].lower() #大文字を小文字に変換する
x_train2 = re.sub(r'\W',' ', x_train2) # アルファベット、アンダーバー、数字以外の文字を空白に置き換える。
x_train2 = re.sub(r'\s+',' ',x_train2) # 垂直タブ以外のすべての空白文字が1回以上繰り返す場合、1個の空白に置き換える。

In [ ]:
x_train[0]

'Full of (then) unknown actors TSF is a great big cuddly romp of a film.<br /><br />The idea of a bunch of bored teenagers ripping off the local sink factory is odd enough, but add in the black humour that Forsyth & Co are so good at and your in for a real treat.<br /><br />The comatose van driver by itself worth seeing, and the canal side chase is just too real to be anything but funny.<br /><br />And for anyone who lived in Glasgow it\'s a great "Oh I know where that is" film.'

In [ ]:
x_train2

'full of then unknown actors tsf is a great big cuddly romp of a film br br the idea of a bunch of bored teenagers ripping off the local sink factory is odd enough but add in the black humour that forsyth co are so good at and your in for a real treat br br the comatose van driver by itself worth seeing and the canal side chase is just too real to be anything but funny br br and for anyone who lived in glasgow it s a great oh i know where that is film '

### 【問題6】Word2Vecの学習

In [ ]:
!pip install gensim

In [ ]:
from gensim.models import Word2Vec
model = Word2Vec(min_count=1, vector_size=10, max_vocab_size=1000) # 次元数を10に設定
model.build_vocab(all_bows3) # 準備
model.train(all_bows3, total_examples=model.corpus_count, epochs=model.epochs) # 学習

(4622, 26615)

In [ ]:
print("語彙の一覧 : {}".format(model.wv.index_to_key))
for vocab in model.wv.index_to_key:
    print("{}のベクトル : \n{}".format(vocab, model.wv[vocab]))

語彙の一覧 : ['e', 'i', 's', 'r', 'n', 't', 'a', 'o', 'l', 'c', 'd', 'u', 'h', 'g', 'm', 'p', 'y', 'b', 'f', 'w', 'v', 'k', 'j', 'x', '9', '0', '1', '5', '3', 'q', 'z', '8', '4', '7', 'á', '2', '6', 'ö', 'é', 'ü']
eのベクトル : 
[-0.13547948 -0.0624392  -0.08455125 -0.14096268  0.2036106   0.1251401
  0.8396069   0.60219246 -0.663359    0.3066738 ]
iのベクトル : 
[-0.08382652 -0.09145691 -0.20257594 -0.20387606  0.30195278  0.21027318
  0.93280303  0.6104743  -0.8013299   0.3126492 ]
sのベクトル : 
[-0.04702578 -0.00603477 -0.05651702 -0.20568937  0.3461376   0.14642026
  0.69283426  0.5283928  -0.63186634  0.2813262 ]
rのベクトル : 
[-0.18619178 -0.06569354 -0.01746122 -0.27076668  0.23845482  0.14995667
  0.74927384  0.38454935 -0.5291256   0.25184512]
nのベクトル : 
[-0.22629571 -0.01645539 -0.22435418 -0.2680216   0.2952995   0.19068922
  0.6894727   0.6019194  -0.55293214  0.4359195 ]
tのベクトル : 
[-0.19282734 -0.01087951 -0.15924288 -0.19105205  0.3456806   0.12379518
  0.71270365  0.38181412 -0.5675674   0.3977